In [13]:
import os
import getpass
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph, START
import functools
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage, HumanMessage

In [3]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"

### Defining Agent State

In [4]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: Annotated[str, operator.setitem]

In [27]:
USER_SYSTEM_PROMPT = """You are a translator. Translate the text provided by the user into {user_language}. Output only the 
    translated text. If the text is already in {user_language}, return the user's text as it is.
"""


class UserAgent(object):

    def __init__(self, llm, prompt):
        self.llm = llm
        self.prompt = prompt
        self.chain = prompt | llm
        self.chat_history = []


    def set_graph(self, graph):
        self.graph = graph

    def send_text(self,text):

        inputs = {"messages": [HumanMessage(content=text)]}
        output = self.graph.invoke(inputs)
        
        return output

    def display_chat_history(self):

        for i in self.chat_history:
            print(i)

    
    def invoke(self, user_text):

        output = self.chain.invoke({'user_text':[user_text]})

        return output
            
        
def create_user_agent(llm, user_language):

    system_prompt = USER_SYSTEM_PROMPT.format(user_language = user_language)
    
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system", system_prompt,
            ),
            MessagesPlaceholder(variable_name = 'user_text'),
        ]
    )

    agent = UserAgent(llm, prompt)

    return agent
    
    
    

In [32]:
french_agent = create_user_agent(llm, "French")
spanish_agent = create_user_agent(llm, "Spanish")
english_agent = create_user_agent(llm, "English")

### Defining Agent Nodes

In [33]:
def user_node(state, agent, name):

    latest_message = state["messages"][-1]
    print(latest_message)
    result = agent.invoke(latest_message.content)
    
    agent.chat_history.append(result.content)

    return {
        "messages": [result],
    }

french_node = functools.partial(user_node, agent=french_agent, name="French")
spanish_node = functools.partial(user_node, agent=spanish_agent, name="Spanish")
english_node = functools.partial(user_node, agent=english_agent, name="Spanish")


In [34]:
def supervisor_node(state):
    print("in supervisor")
    print(state)
    
    return {'messages': [],
           # 'sender':"Supervisor",
           } 

In [35]:
workflow = StateGraph(AgentState)

agentList = {'French':{'node':french_node, 'agent':french_agent},
                   'Spanish':{'node':spanish_node, 'agent':spanish_agent},
                   'English':{'node':english_node, 'agent':english_agent},
                   }
                  

#Defining nodes
for agent in agentList:
    workflow.add_node(agent, agentList[agent]['node'])

workflow.add_node("Supervisor", supervisor_node)

#Defining edges
workflow.add_edge(START, "Supervisor")

for agent in agentList:
    workflow.add_edge("Supervisor", agent)
    workflow.add_edge(agent, END)



In [36]:
app = workflow.compile()

In [37]:
french_agent.set_graph(app)
spanish_agent.set_graph(app)
english_agent.set_graph(app)

In [38]:
english_agent.send_text("hey there!")

in supervisor
{'messages': [HumanMessage(content='hey there!')], 'sender': None}
content='hey there!'
content='hey there!'
content='hey there!'


{'messages': [HumanMessage(content='hey there!'),
  AIMessage(content='hey là-bas!', response_metadata={'finish_reason': 'stop'}, id='run-e5819d2c-ef73-448f-9591-0d6c80dc025f-0'),
  AIMessage(content='¡Hola!', response_metadata={'finish_reason': 'stop'}, id='run-4464ac30-8c50-4c5a-926f-4f6e158d3c0e-0'),
  AIMessage(content='hey there!', response_metadata={'finish_reason': 'stop'}, id='run-a65b46a1-7100-4794-8018-382eb19699a6-0')]}

In [39]:
english_agent.send_text("How are you?")

in supervisor
{'messages': [HumanMessage(content='How are you?')], 'sender': None}
content='How are you?'
content='How are you?'
content='How are you?'


{'messages': [HumanMessage(content='How are you?'),
  AIMessage(content='Comment vas-tu ?', response_metadata={'finish_reason': 'stop'}, id='run-62452744-032f-4a7a-b960-d275ee4dcc3b-0'),
  AIMessage(content='¿Cómo estás?', response_metadata={'finish_reason': 'stop'}, id='run-847b6142-cae7-4ec4-a1c8-3c1ebe2a3ff4-0'),
  AIMessage(content='How are you?', response_metadata={'finish_reason': 'stop'}, id='run-d1a2a68f-fb67-4882-b053-5dcd62dbb264-0')]}

In [40]:
spanish_agent.send_text("lo estoy haciendo bien. ¿Y tú?")

in supervisor
{'messages': [HumanMessage(content='lo estoy haciendo bien. ¿Y tú?')], 'sender': None}
content='lo estoy haciendo bien. ¿Y tú?'
content='lo estoy haciendo bien. ¿Y tú?'
content='lo estoy haciendo bien. ¿Y tú?'


{'messages': [HumanMessage(content='lo estoy haciendo bien. ¿Y tú?'),
  AIMessage(content='Je le fais bien. Et toi?', response_metadata={'finish_reason': 'stop'}, id='run-0404cc91-b7fc-465a-bba7-c635dcb1a817-0'),
  AIMessage(content="I'm doing well. And you?", response_metadata={'finish_reason': 'stop'}, id='run-691655aa-49bf-4fd4-b472-8293f85bfb41-0'),
  AIMessage(content="I'm doing well. And you?", response_metadata={'finish_reason': 'stop'}, id='run-99668afc-8f36-49b2-8a45-a027e04729aa-0')]}

In [41]:
french_agent.send_text("tu viens au bureau aujourd'hui ?")

in supervisor
{'messages': [HumanMessage(content="tu viens au bureau aujourd'hui ?")], 'sender': None}
content="tu viens au bureau aujourd'hui ?"
content="tu viens au bureau aujourd'hui ?"
content="tu viens au bureau aujourd'hui ?"


{'messages': [HumanMessage(content="tu viens au bureau aujourd'hui ?"),
  AIMessage(content='Are you coming to the office today?', response_metadata={'finish_reason': 'stop'}, id='run-397fd6a5-4f28-4051-b750-53735dcf2576-0'),
  AIMessage(content='¿Vas a venir a la oficina hoy?', response_metadata={'finish_reason': 'stop'}, id='run-15e3a62a-b7d9-4c8c-ac03-751a5907f57d-0'),
  AIMessage(content='Are you coming to the office today?', response_metadata={'finish_reason': 'stop'}, id='run-49669e85-e1d8-4461-bc05-b0d8288a8a02-0')]}

In [42]:
spanish_agent.display_chat_history()

¡Hola!
¿Cómo estás?
I'm doing well. And you?
¿Vas a venir a la oficina hoy?


In [43]:
english_agent.display_chat_history()

hey there!
How are you?
I'm doing well. And you?
Are you coming to the office today?


In [44]:
french_agent.display_chat_history()

hey là-bas!
Comment vas-tu ?
Je le fais bien. Et toi?
Are you coming to the office today?
